<a href="https://colab.research.google.com/github/RM-RAMASAMY/exploring_timegpt_tabula9_and_relational_deep_learning/blob/main/research_on_tabular_foundation_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# optional setup; use if the notebook is not running inside the rtfm conda environment
!git clone https://github.com/mlfoundations/rtfm.git
%cd rtfm

# Ensure pip is up to date
!pip install --upgrade pip

# Install Python 3.8 using pip
!pip install python==3.8

# Install pip dependencies from requirements.txt
!pip install -r requirements.txt

# Install additional dependencies
!pip install git+https://github.com/jpgard/llama-recipes.git
!pip install -e .
!pip install --no-deps git+https://github.com/mlfoundations/tableshift.git

fatal: destination path 'rtfm' already exists and is not an empty directory.
/content/rtfm
ERROR: Could not find a version that satisfies the requirement python==3.8 (from versions: none)
ERROR: No matching distribution found for python==3.8
  Cloning https://github.com/jpgard/llama-recipes.git to /tmp/pip-req-build-vxogvt75
  Running command git clone --filter=blob:none --quiet https://github.com/jpgard/llama-recipes.git /tmp/pip-req-build-vxogvt75
  Resolved https://github.com/jpgard/llama-recipes.git to commit 186213f2ba3382f0aaa29326377e36c840d78160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Obtaining file:///content/rtfm
  Preparing metadata (setup.py) ... done
  Attempting uninstall: rtfm
    Found existing installation: rtfm 0.0
    Uninstalling rtfm-0.0:
      Successfully uninstalled rtfm-0.0
  DEPRECATION: Legacy editable install of rtfm==0.0 from file:///content/rtfm (setup.py develop

In [6]:
!pip install git+https://github.com/mlfoundations/tabliblib.git

  Cloning https://github.com/mlfoundations/tabliblib.git to /tmp/pip-req-build-d7y8h92u
  Running command git clone --filter=blob:none --quiet https://github.com/mlfoundations/tabliblib.git /tmp/pip-req-build-d7y8h92u
  Resolved https://github.com/mlfoundations/tabliblib.git to commit 95f5ae40227da9f3083852e7fcdfd2e4b31b2d51
  Preparing metadata (setup.py) ... done
  Created wheel for tabliblib: filename=tabliblib-0.0-py3-none-any.whl size=30585 sha256=736fdf400c6fb1c97e291daafe7df15557c0e236e1e3a478dd41bc226d50f7a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-gldy_epl/wheels/b0/c1/44/e7ffba34c360d2e8e56073c2ebe52499fa60274b2e7986956b
Successfully built tabliblib


In [8]:
!pip install nltk
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [13]:
!pip install tablib
import pandas as pd
import torch
from transformers import AutoTokenizer, LlamaForCausalLM, AutoConfig
import tablib # Import tablib

from rtfm.configs import TrainConfig, TokenizerConfig
from rtfm.inference_utils import InferenceModel
from rtfm.serialization.serializers import get_serializer
from rtfm.tokenization.text import prepare_tokenizer

train_config = TrainConfig(model_name="mlfoundations/tabula-8b", context_length=8192)

# If using a base llama model (not fine-tuned TabuLa),
# make sure to set add_serializer_tokens=False
# (because we do not want to use special tokens for
# the base model which is not trained on them).
tokenizer_config = TokenizerConfig()

# Load the configuration
config = AutoConfig.from_pretrained(train_config.model_name)

# Set the torch_dtype to bfloat16 which matches TabuLa train/eval setup
config.torch_dtype = 'bfloat16'

device = "cuda" if torch.cuda.is_available() else "cpu"

model = LlamaForCausalLM.from_pretrained(
    train_config.model_name, device_map="auto", config=config, offload_folder=None)

tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
serializer = get_serializer(tablib) # Now tablib is defined

tokenizer, model = prepare_tokenizer(
    model,
    tokenizer=tokenizer,
    pretrained_model_name_or_path=train_config.model_name,
    model_max_length=train_config.context_length,
    use_fast_tokenizer=tokenizer_config.use_fast_tokenizer,
    serializer_tokens_embed_fn=tokenizer_config.serializer_tokens_embed_fn,
    serializer_tokens=serializer.special_tokens
    if tokenizer_config.add_serializer_tokens
    else None,
)

inference_model = InferenceModel(model=model, tokenizer=tokenizer, serializer=serializer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


AttributeError: module 'tablib' has no attribute 'serializer_cls'

In [ ]:
labeled_examples = pd.DataFrame(
    [
        {"location": "New York", "temperature": 22, "humidity": 65, "wind_speed": 12, "pressure": 1012, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Partly Sunny"},
        {"location": "Los Angeles", "temperature": 26, "humidity": 60, "wind_speed": 7, "pressure": 1015,
         "month": "July", "weather_yesterday": "Partly Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
        {"location": "Chicago", "temperature": 18, "humidity": 70, "wind_speed": 15, "pressure": 1008, "month": "July",
         "weather_yesterday": "Partly Cloudy", "precipitation": 0.1, "visibility": 8, "weather_today": "Cloudy"},
        {"location": "Houston", "temperature": 30, "humidity": 80, "wind_speed": 10, "pressure": 1010, "month": "July",
         "weather_yesterday": "Rain", "precipitation": 0.5, "visibility": 7, "weather_today": "Rain"},
        {"location": "Phoenix", "temperature": 35, "humidity": 20, "wind_speed": 5, "pressure": 1005, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
        {"location": "Philadelphia", "temperature": 24, "humidity": 75, "wind_speed": 14, "pressure": 1009,
         "month": "July", "weather_yesterday": "Partly Cloudy", "precipitation": 0.2, "visibility": 9,
         "weather_today": "Partly Cloudy"},
        {"location": "San Antonio", "temperature": 28, "humidity": 68, "wind_speed": 11, "pressure": 1011,
         "month": "July", "weather_yesterday": "Rain", "precipitation": 0.4, "visibility": 8, "weather_today": "Rain"},
        {"location": "San Diego", "temperature": 22, "humidity": 65, "wind_speed": 10, "pressure": 1014,
         "month": "July", "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Partly Sunny"},
        {"location": "Dallas", "temperature": 27, "humidity": 72, "wind_speed": 9, "pressure": 1007, "month": "July",
         "weather_yesterday": "Partly Cloudy", "precipitation": 0.3, "visibility": 9, "weather_today": "Cloudy"},
    ]
)
target_example = pd.DataFrame(
    [
        {"location": "San Jose", "temperature": 23, "humidity": 55, "wind_speed": 8, "pressure": 1013, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
    ]
)

output = inference_model.predict(
    target_example=target_example,
    target_colname="weather_today",
    target_choices=["Sunny", "Partly Sunny", "Cloudy", "Partly Cloudy", "Rain"],
    labeled_examples=labeled_examples,
)
print(f"Prediction for sample \n {target_example} \n is: {output}")

In [ ]:
from rtfm.serialization.serialization_utils import discretize_continuous_column

examples = pd.DataFrame(
    [
    {"location": "New York", "size_sqft": 1200, "bedrooms": 3, "bathrooms": 2, "age": 10, "lot_size_acres": 0.15, "garage": True, "price": 850},
    {"location": "Los Angeles", "size_sqft": 1500, "bedrooms": 4, "bathrooms": 3, "age": 8, "lot_size_acres": 0.25, "garage": True, "price": 950},
    {"location": "Chicago", "size_sqft": 1300, "bedrooms": 3, "bathrooms": 2, "age": 15, "lot_size_acres": 0.2, "garage": False, "price": 700},
    {"location": "Houston", "size_sqft": 1700, "bedrooms": 4, "bathrooms": 3, "age": 5, "lot_size_acres": 0.3, "garage": True, "price": 650},
    {"location": "Phoenix", "size_sqft": 1600, "bedrooms": 3, "bathrooms": 2, "age": 7, "lot_size_acres": 0.25, "garage": True, "price": 750},
    {"location": "Philadelphia", "size_sqft": 1400, "bedrooms": 3, "bathrooms": 2, "age": 12, "lot_size_acres": 0.18, "garage": False, "price": 600},
    {"location": "San Antonio", "size_sqft": 1800, "bedrooms": 4, "bathrooms": 3, "age": 3, "lot_size_acres": 0.4, "garage": True, "price": 700},
    {"location": "San Diego", "size_sqft": 1550, "bedrooms": 3, "bathrooms": 2, "age": 9, "lot_size_acres": 0.22, "garage": True, "price": 850},
    {"location": "Dallas", "size_sqft": 1450, "bedrooms": 3, "bathrooms": 2, "age": 11, "lot_size_acres": 0.19, "garage": True, "price": 700},
    {"location": "San Jose", "size_sqft": 1600, "bedrooms": 4, "bathrooms": 3, "age": 6, "lot_size_acres": 0.2, "garage": False, "price": 800},
    {"location": "Seattle", "size_sqft": 1800, "bedrooms": 4, "bathrooms": 2, "age": 10, "lot_size_acres": 0.2, "garage": False, "price": 925},
]
)

examples["price"] = discretize_continuous_column(examples["price"], num_buckets=4)
target_choices = examples["price"].unique().tolist()

target_example = examples.iloc[[0]]
labeled_examples = examples.iloc[1:]


output = inference_model.predict(
    target_example=target_example,
    target_colname="price",
    target_choices=target_choices,
    labeled_examples=labeled_examples,
)
print(f"Prediction for sample \n {target_example} \n is: {output}")